In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
import tigre

In [3]:
# Load the mat files and extract the data
import scipy.io
import os

# Specify the directory where the mat files are located
directory = '/media/jericho/T7/Jericho_MAR_Studies/'

# Get the list of all the mat files in the directory
files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.mat')]
files.sort()

# Load the mat files into a pandas dataframe
data = []
for file in files:
    data.append(scipy.io.loadmat(file))


In [4]:

# Extract the data from the mat files into a pandas dataframe
# dict_keys(['__header__', '__version__', '__globals__', 'L', 'metadata'])
data_all = {}
for i in range(len(data)):
    f_name = files[i].split('/')[-1]
    data_all[f_name] = {}
    data_all[f_name]['projections'] = data[i]['L']
    data_all[f_name]['metadata'] = data[i]['metadata']


In [45]:
first_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Al kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
first_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Al kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']
secon_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Al kV-CBCT DLI L2 140kVp 25mAs.mat']['projections']
secon_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Al kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']

In [32]:
first_layer_bilat_proj_nm = data_all['2024_01_20 CIRS801PF Pelvis NoMetal kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
first_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']
secon_layer_bilat_proj_nm = data_all['2024_01_20 CIRS801PF Pelvis NoMetal kV-CBCT DLI L2 140kVp 25mAs.mat']['projections']
secon_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['metadata']

In [33]:
# first_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Spine Metal_DoubleScrew kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
# first_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']
# secon_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Spine Metal_DoubleScrew kV-CBCT DLI L2 140kVp 25mAs.mat']['projections']
# secon_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['metadata']

In [34]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter, disk
from skimage.util import img_as_ubyte
from skimage.restoration import inpaint
from skimage.morphology import binary_dilation

def inpaint_images_spine(images0,images1):
    results = []
    images2 = images0.copy()
    images3 = images1.copy()
    
    for image_high, image_low in zip(images2, images3):

        ratio0 = image_high/(image_low)

        mask = np.zeros_like(image_high)
        
        mask[240:,125:145 ] = ratio0[240:,125:145 ] > 1.05
        # mask[240:,125:145 ] += image_high[240:,125:145 ] > p_98
        # mask[240:290,125:145 ] += ratio0[240:290,125:145 ]*image_high[240:290,125:145 ] > 6.5*1.1
        mask[mask == 2] = 1
        mask[mask == 3] = 1
        image_high[mask.astype(bool)] = np.NaN
        # image_high[image_high > p_98] = np.NaN
        # Make the values above below and to the side of the NaNs NaN
        # mask = np.isnan(image_high)
        expanded_mask = binary_dilation(mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        expanded_mask = binary_dilation(expanded_mask)

        image_high[expanded_mask] = np.NaN
        # Inpaint image
        mask =  np.isnan(image_high)
        image_result = inpaint.inpaint_biharmonic(image_high, mask)
        results.append(image_result)
        # results.append(image_high)
    return results
images_LI = inpaint_images_spine(first_layer_bilat_proj.T[:],first_layer_bilat_proj_nm.T[:])

/tmp/ipykernel_49809/4218207276.py:19: RuntimeWarning: divide by zero encountered in divide
  ratio0 = image_high/(image_low)
/tmp/ipykernel_49809/4218207276.py:19: RuntimeWarning: invalid value encountered in divide
  ratio0 = image_high/(image_low)


In [35]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter, disk
from skimage.util import img_as_ubyte
from skimage.restoration import inpaint
from skimage.morphology import binary_dilation

def inpaint_images_spine(images0,images1):
    results = []
    images2 = images0.copy()
    images3 = images1.copy()
    
    for image_high, image_low in zip(images2, images3):

        ratio0 = image_high/(image_low)

        mask = np.zeros_like(image_high)
        mask[240:,125:145 ] = ratio0[240:,125:145 ] > 1.05
        # mask[240:,125:145 ] += image_high[240:,125:145 ] > p_98
        # mask[240:290,125:145 ] += ratio0[240:290,125:145 ]*image_high[240:290,125:145 ] > 6.5*1.1
        mask[mask == 2] = 1
        mask[mask == 3] = 1
        image_high[mask.astype(bool)] = np.NaN
        # image_high[image_high > p_98] = np.NaN
        # Make the values above below and to the side of the NaNs NaN
        # mask = np.isnan(image_high)
        expanded_mask = binary_dilation(mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        expanded_mask = binary_dilation(expanded_mask)

        image_high[expanded_mask] = np.NaN
        # Inpaint image
        mask =  np.isnan(image_high)
        image_result = inpaint.inpaint_biharmonic(image_high, mask)
        results.append(image_result)
        # results.append(image_high)
    return results

images_LI_second = inpaint_images_spine(secon_layer_bilat_proj.T[:],first_layer_bilat_proj_nm.T[:])

/tmp/ipykernel_49809/4049590518.py:19: RuntimeWarning: divide by zero encountered in divide
  ratio0 = image_high/(image_low)
/tmp/ipykernel_49809/4049590518.py:19: RuntimeWarning: invalid value encountered in divide
  ratio0 = image_high/(image_low)


In [36]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter, disk
from skimage.util import img_as_ubyte
from skimage.restoration import inpaint
from skimage.morphology import binary_dilation

def trace_replacement_spine(images0,images1,images4):
    results = []
    images2 = images0.copy()
    images3 = images1.copy()
    images4 = images4.copy()
    
    for image_high, image_low, image_low2 in zip(images2, images3, images4):

        ratio0 = image_high/(image_low)

        mask = np.zeros_like(image_high)
        mask[240:,125:145 ] = ratio0[240:,125:145 ] > 1.05
        # mask[240:,125:145 ] += image_high[240:,125:145 ] > p_98
        # mask[240:290,125:145 ] += ratio0[240:290,125:145 ]*image_high[240:290,125:145 ] > 6.5*1.1
        mask[mask == 2] = 1
        mask[mask == 3] = 1
        image_high[mask.astype(bool)] = np.NaN
        # image_high[image_high > p_98] = np.NaN
        # Make the values above below and to the side of the NaNs NaN
        # mask = np.isnan(image_high)
        expanded_mask = binary_dilation(mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        expanded_mask = binary_dilation(expanded_mask)

        image_high[expanded_mask] = np.NaN
        # Inpaint image
        mask =  np.isnan(image_high)
        # image_result = inpaint.inpaint_biharmonic(image_high, mask)
        # results.append(image_result)
        image_high[mask] = image_low2[mask]
        results.append(image_high)
    return results

images_LI_tr = trace_replacement_spine(first_layer_bilat_proj.T[:],0.25*first_layer_bilat_proj.T,first_layer_bilat_proj_nm.T[:])

/tmp/ipykernel_49809/2089553179.py:20: RuntimeWarning: invalid value encountered in divide
  ratio0 = image_high/(image_low)


In [37]:
geo = tigre.geometry_default()

In [46]:
# plt.figure()
# plt.plot(secon_layer_bilat_header[0][-1][2][1,:], secon_layer_bilat_header[0][-1][2][0,:], 'o')
# plt.plot(first_layer_bilat_header[0][-1][1][1,:], first_layer_bilat_header[0][-1][1][0,:], 'o')

source_pos_x = secon_layer_bilat_header[0][-1][2][1,:]
source_pos_y = secon_layer_bilat_header[0][-1][2][0,:]
detector_pos_x = first_layer_bilat_header[0][-1][1][1,:]
detector_pos_y = first_layer_bilat_header[0][-1][1][0,:]

# calculate the angles from the source to the detector
angles_real = np.arctan2(detector_pos_x - source_pos_x, detector_pos_y - source_pos_y)

# calculate the distance from the source to the detector
distance = np.sqrt((detector_pos_x - source_pos_x)**2 + (detector_pos_y - source_pos_y)**2)


In [47]:
import h5py
import numpy as np

class Geo:
    pass

def assign_values_to_geo(filename):
    geo = Geo()
    with h5py.File(filename, 'r') as f:
        def assign_value(name, obj):
            if isinstance(obj, h5py.Dataset):
                # Remove 'geo/' from the name and replace any remaining '/' with '_'
                attr_name = name.replace('geo/', '').replace('/', '_')
                setattr(geo, attr_name, np.array(obj[0][()]))
        f.visititems(assign_value)
    return geo

# Use the function
geo2 = assign_values_to_geo('/home/jericho/Downloads/firefox-downloads/geo.mat')

# Now you can access the values like this:
print(geo2.DSD)
print(geo2.offDetector)
# print_hdf5_file_structure()

[1500.]
[160.   0.]


In [48]:
geo.DSD = 1527.3
geo.DSO = 1000
# geo.mode = geo2.mode
geo.nDetector = np.array([266, 354])
geo.dDetector = np.array([1.12,1.12])
geo.sDetector = geo.nDetector*geo.dDetector
geo.offDetector = np.array([0,-160])# geo2.offDetector
geo.nVoxel = np.array([120,512,512])
geo.dVoxel = np.array([1.2,1,1])
geo.sVoxel = geo.nVoxel*geo.dVoxel
geo.offOrigin = geo2.offOrigin

In [49]:
angles = angles_real 
# Shift the angles by 110 degrees 
angles = angles + np.deg2rad(-90)
# Make sure that the angles are between 0 and 2*pi
angles = np.mod(angles, 2*np.pi)


In [51]:
img_first = tigre.algorithms.fdk(first_layer_bilat_proj.transpose([2,0,1]),geo,angles,dowang=True)
img_second = tigre.algorithms.fdk(secon_layer_bilat_proj.transpose([2,0,1]),geo,angles,dowang=True)
img_LI = tigre.algorithms.fdk(np.array(images_LI).transpose([0,2,1])[:-1],geo,angles,dowang=True)


FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(895, 266, 639)
FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
D

IndexError: index 893 is out of bounds for axis 0 with size 893

In [52]:

img_gt = tigre.algorithms.fdk(first_layer_bilat_proj_nm.transpose([2,0,1]),geo,angles[:-1],dowang=True)
# img_combined = tigre.algorithms.fdk(np.array(images_LI).transpose([0,2,1]),geo,angles,dowang=True)

FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(894, 266, 639)


In [65]:
images = []

for ii in np.linspace(0,1.2,10):
    im = ii*np.log(first_layer_bilat_proj.T[100]) - np.log(secon_layer_bilat_proj.T[100])
    im[im == -np.inf] = 0
    im[im == np.inf] = 0
    im[np.isnan(im)] = 0
    images.append(im)

    

/tmp/ipykernel_49809/2002481649.py:4: RuntimeWarning: divide by zero encountered in log
  im = ii*np.log(first_layer_bilat_proj.T[100]) - np.log(secon_layer_bilat_proj.T[100])
/tmp/ipykernel_49809/2002481649.py:4: RuntimeWarning: invalid value encountered in multiply
  im = ii*np.log(first_layer_bilat_proj.T[100]) - np.log(secon_layer_bilat_proj.T[100])
/tmp/ipykernel_49809/2002481649.py:4: RuntimeWarning: invalid value encountered in subtract
  im = ii*np.log(first_layer_bilat_proj.T[100]) - np.log(secon_layer_bilat_proj.T[100])


In [67]:
plt.figure()
pli.hyperslicer(images,vmin=0.1,vmax=0.5)

AttributeError: 'Controls' object has no attribute 'vbox'

In [54]:
img_tr = tigre.algorithms.fdk(np.array(images_LI_tr).transpose([0,2,1]),geo,angles[:-1],dowang=True)
# img_wtr = tigre.algorithms.fdk(np.array(images_LI_wtr).transpose([0,2,1]),geo,angles,dowang=True)
img_LI_second = tigre.algorithms.fdk(np.array(images_LI_second).transpose([0,2,1]),geo,angles[:-1],dowang=True)

FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(894, 266, 639)
FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
D

In [55]:
def mask_center(image, radius):
    # Get the center of the image
    center = np.array(image.shape) / 2

    # Create an array of the same shape as the image, filled with the squared distance from the center
    y, x = np.ogrid[:image.shape[0], :image.shape[1]]
    dist_from_center = (x - center[1])**2 + (y - center[0])**2

    # Create a mask that is True for all pixels within the specified radius of the center
    mask = dist_from_center <= radius**2

    # Use the mask to set the corresponding pixels in the original image to 0
    image[~mask] = 0

    return image

# # crop the image from 512 to 416
# img = img[:,48:464, 48:464]
# # Test the function
# image = np.ones((100, 100))
radius = 416//2
# for im in img:
#     im = mask_center(im, radius)

img_LI0 = img_LI[:,48:464, 48:464].copy()
for im in img_LI0:
    im = mask_center(im, radius)

# img[img < 0.01] = 0
img_LI0[img_LI0 < 0.01] = 0

# do the same for the first and second layers

img_first0 = img_first[:,48:464, 48:464].copy()
for im in img_first0:
    im = mask_center(im, radius)

img_second0 = img_second[:,48:464, 48:464].copy()
for im in img_second0:
    im = mask_center(im, radius)

img_gt0 = img_gt[:,48:464, 48:464].copy()
for im in img_gt0:
    im = mask_center(im, radius)

img_tr0 = img_tr[:,48:464, 48:464].copy()
for im in img_tr0:
    im = mask_center(im, radius)

# img_wtr0 = img_wtr[:,48:464, 48:464].copy()
# for im in img_wtr0:
#     im = mask_center(im, radius)

img_LI_second0 = img_LI_second[:,48:464, 48:464].copy()
for im in img_LI_second0:
    im = mask_center(im, radius)

img_first0[img_first0 < 0.01] = 0
img_first0[img_first0 < 0.01] = 0
img_second0[img_second0< 0.01] = 0
img_gt0[img_gt0 < 0.01] = 0
img_tr0[img_tr0 < 0.01] = 0
# img_wtr0[img_wtr0 < 0.01] = 0
img_LI_second0[img_LI_second0 < 0.01] = 0

# convert each of the images to HU using the water and air values
water = 0.0182
air = 0
img_LI0 = (img_LI0 - air)/(water - air)*1000 - 1000
img_first0 = (img_first0 - air)/(water - air)*1000 - 1000
img_second0 = (img_second0 - air)/(water - air)*1000    - 1000
img_gt0 = (img_gt0 - air)/(water - air)*1000 - 1000
img_tr0 = (img_tr0 - air)/(water - air)*1000 - 1000
# img_wtr0 = (img_wtr0 - air)/(water - air)*1000 - 1000
img_LI_second0 = (img_LI_second0 - air)/(water - air)*1000 - 1000

plt.figure()
plt.subplot(3,2,1)
plt.imshow(img_first0[62],vmin=-400,vmax=400,cmap='gray')
plt.title('First Layer')
plt.axis('off')
plt.subplot(3,2,2)
plt.imshow(img_second0[62],vmin=-400,vmax=400,cmap='gray')
plt.title('Second Layer')
plt.axis('off')
plt.subplot(3,2,3)
plt.imshow(img_LI0[62],vmin=-400,vmax=400,cmap='gray')
plt.title('Biharmonic Inpainting')
plt.axis('off')
plt.subplot(3,2,4)
plt.imshow(img_gt0[62],vmin=-400,vmax=400,cmap='gray')
plt.title('Ground Truth')
plt.axis('off')
plt.subplot(3,2,5)
plt.imshow(img_tr0[62],vmin=-400,vmax=400,cmap='gray')
plt.title('Trace Replacement')
plt.axis('off')
plt.subplot(3,2,6)
plt.imshow(img_LI_second0[62],vmin=-400,vmax=400,cmap='gray')
plt.title('Biharmonic Inpainting Second Layer')
plt.axis('off')

(-0.5, 415.5, 415.5, -0.5)

In [66]:
# plt.figure()

# for ii, alpha in enumerate([0.1,0.5,1.0,2.0]):
#     img_sub = tigre.algorithms.fdk((first_layer_bilat_proj - alpha*secon_layer_bilat_proj).transpose([2,0,1]),geo,angles,dowang=True)
#     img_sub0 = img_sub[:,48:464, 48:464].copy()
#     # for im in img_sub0:
#     #     im = mask_center(im, radius)
#     # img_sub0[img_sub0 < 0.01] = 0
#     # img_sub0 = (img_sub0 - air)/(water - air)*1000 - 1000
#     plt.subplot(2,2,ii+1)
#     plt.imshow(img_sub0[62],cmap='gray')
#     plt.title('Subtraction with alpha = ' + str(alpha))
#     plt.axis('off')

img_sub = tigre.algorithms.fdk((first_layer_bilat_proj - 2*secon_layer_bilat_proj).transpose([2,0,1]),geo,angles,dowang=True)

FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(894, 266, 639)


In [68]:
images = []

for coeff in np.linspace(0,100,100):
    images.append(img_first[62] - img_sub[62]*(coeff/5))

plt.figure()
cc = pli.hyperslicer(images)#,vmin=0,vmax=0.05)

In [62]:
plt.figure()
plt.imshow(10*img_sub[62]-img_first[62],cmap='gray')

In [18]:
# load an h5py  file
import h5py
import numpy as np

def print_hdf5_file_structure(file_name):
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print("    %s: %s" % (key, val))

    with h5py.File(file_name, 'r') as f:
        f.visititems(print_attrs)

fname = '/home/jericho/Software/dl_mar/OSCNet/data/test/test_640geo/000376_02_01/040/0.h5'

print_hdf5_file_structure(fname)

LI_CT
LI_sinogram
ma_CT
ma_sinogram
metal_trace


In [28]:
def test_image(data_path, imag_idx, mask_idx):
    txtdir = os.path.join(data_path, 'test_640geo_dir.txt')
    mat_files = open(txtdir, 'r').readlines()
    gt_dir = mat_files[imag_idx]
    file_dir = gt_dir[:-6]
    data_file = file_dir + str(mask_idx) + '.h5'
    abs_dir = os.path.join(data_path, 'test_640geo/', data_file)
    gt_absdir = os.path.join(data_path, 'test_640geo/', gt_dir[:-1])
    gt_file = h5py.File(gt_absdir, 'r')
    Xgt = gt_file['image'][()]
    gt_file.close()
    file = h5py.File(abs_dir, 'r')
    Xma= file['ma_CT'][()]
    XLI =file['LI_CT'][()]
    file.close()

    return Xgt, Xma, XLI
    # M512 = test_mask[:,:,mask_idx]
data_path = '/home/jericho/Software/dl_mar/OSCNet/data/test'
Xgt, Xma, XLI = test_image(data_path, 0, 0)


In [30]:
plt.figure()
pli.hyperslicer([Xgt,Xma,XLI])

AttributeError: 'Controls' object has no attribute 'vbox'

In [26]:
plt.figure()
plt.hist(Xma.flatten(), histtype='step',bins=100, alpha=0.5, label='ma_ct',log=True)
plt.hist(XLI.flatten(), histtype='step',bins=100, alpha=0.5, label='li_ct',log=True)
plt.hist(img[62].flatten(), histtype='step',bins=100, alpha=0.5, label='fdk_ma_ct',log=True)
plt.hist(img_LI[62].flatten(), histtype='step',bins=100, alpha=0.5, label='fdk_li_ct',log=True)
plt.legend()

In [27]:
# Open 0.h5 replace Xma with the img[55] and save as h5py file 1.h5
import h5py

fname2 = '/home/jericho/Software/dl_mar/OSCNet/data/test/test_640geo/000376_02_01/040/0.h5'

# Load the file
import h5py
import numpy as np

# Open the file in 'r+' mode
with h5py.File(fname2, 'r+') as f:
    # Load the images 
    f['ma_CT'][()] = img[62]
    f['LI_CT'][()] = img_LI[62]
    # The changes are automatically saved when the file is closed

In [94]:
%matplotlib qt

#%% DEMO 14:  Playing with offsets
#
#
# In this demo we show how to change offsets to either the iamge or the
# detector, and the flexibility of it.
#
#
# --------------------------------------------------------------------------
# --------------------------------------------------------------------------
# This file is part of the TIGRE Toolbox
#
# Copyright (c) 2015, University of Bath and
#                     CERN-European Organization for Nuclear Research
#                     All rights reserved.
#
# License:            Open Source under BSD.
#                     See the full license at
#                     https://github.com/CERN/TIGRE/blob/master/LICENSE
#
# Contact:            tigre.toolbox@gmail.com
# Codes:              https://github.com/CERN/TIGRE/
# Coded by:           Ander Biguri
# --------------------------------------------------------------------------
## Initialize
#%%Initialize
import tigre
import numpy as np
from tigre.utilities import sample_loader
import matplotlib.pyplot as plt
import tigre.algorithms as algs

#%% Geometry
geo = tigre.geometry_default(high_resolution=False)


# Offsets
## ####################################################################################
# Lets try simnple offset: The detector gets completelly displaced
geo.offOrigin = np.array([0, 0, 0])  # Offset of image from origin   (mm)
geo.offDetector = np.array([200, 200])  # Offset of Detector            (mm)
## ####################################################################################
# Auxiliary
geo.accuracy = 0.5
# Accuracy of FWD proj          (vx/sample)

## Load data and generate projections
# see previous demo for explanation
angles = np.linspace(0, 2 * np.pi, 100)

# Load thorax phatom data
head = sample_loader.load_head_phantom(geo.nVoxel)
projections = tigre.Ax(head, geo, angles)


#%% lets see it
# tigre.plotproj(projections, angles)

## we will skip reconstruction of this tests because the image is outside the detector


## #####################################################################

#%% Second test: lets test variying offsets:
geo.nDetector = np.array([80,120])  # Number of detector pixels
geo.dDetector = np.array([4, 4])  # Detector element size (mm)
geo.sDetector = geo.nDetector * geo.dDetector  # Detector size (mm)
geo.offDetector = np.array(
    [0,-160]
)  # Offset of Detector            (mm)
projections2 = tigre.Ax(head, geo, angles)
## lets see it
# tigre.plotproj(projections2, angles)
## reconstruction
geo.offDetector = np.array(
    [0,-160]
)  

res = algs.fdk(projections2, geo, angles, dowang=True)


# tigre.plotimg(res, dim="z")

#%% Third test: lets vary everything
# plt.show()

FDK: applying detector offset weights
-80 200 0 -80.0
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1536.0 mm
Distance from source to origin (DSO)= 1000.0 mm
-----
Detector parameters
Number of pixels (nDetector) = [ 80 200]
Size of each pixel (dDetector) = [4 4] mm
Total size of the detector (sDetector) = [320 800] mm
-----
Image parameters
Number of voxels (nVoxel) = [64 64 64]
Total size of the image (sVoxel) = [256 256 256] mm
Size of each voxel (dVoxel) = [4. 4. 4.] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0 0 0] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
-----
Rotation of the Detector (rotDetector) = [0 0 0] rad
(100, 80, 200)


In [86]:
# load a png file and plot it using imshow
png_file = '/home/jericho/Software/dl_mar/OSCNet/save_results/osc/1.png'
import mpl_interactions.ipyplot as iplt
img = plt.imread(png_file)
plt.figure()
cc = iplt.imshow(img.mean(axis=2),vmin_vmax=(0,1))

In [96]:
plt.figure()
pli.hyperslicer(res)

AttributeError: 'Controls' object has no attribute 'vbox'

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import tigre

In [8]:
import mpl_interactions as pli


plt.figure()
# pli.hyperslicer(first_layer_bilat_proj.transpose([2,0,1]))
pli.hyperslicer(projections2)

AttributeError: 'Controls' object has no attribute 'vbox'